# Polygon.io Example Notebooks
In these examples, we will leverage `polygon-api-client`. To ensure the examples run with no code changes, put your api key in the `POLYGON_API_KEY` environment variable. You can do this from python by running the following before any other cells:

```python
import os
os.environ["POLYGON_API_KEY"] = "..."  # Your api key here
```

## Example - Building a company profile widget with ipywidgets
In this example, we'll build an interactive company profile widget with [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/).


In [78]:
# imports

# python standard library
import os
from datetime import datetime, timedelta
from io import BytesIO

# Polygon Rest Client
from polygon import RESTClient

# Pandas
import pandas as pd

# ipywidgets
import ipywidgets as ipy

# requests
import requests

In [47]:
# Instantiate client
client = RESTClient(os.environ["POLYGON_API_KEY"])

In [49]:
# First, lets build an autocomplete widget
# instantiate a combobox
combo = ipy.Combobox()

def lookup(change):
    results = client.reference_tickers_v3(search=change["new"].upper(), limit=50).results
    combo.options = [r["ticker"] for r in results]

combo.observe(lookup, "value")
combo

Combobox(value='')

In [112]:
# Now lets build a little profile widget

# First lets grab a sample result
ticker = "AAPL"
key = os.environ["POLYGON_API_KEY"]
data = client.reference_ticker_details_vx(ticker).results
data

{'ticker': 'AAPL',
 'name': 'Apple Inc.',
 'market': 'stocks',
 'locale': 'us',
 'primary_exchange': 'XNAS',
 'type': 'CS',
 'active': True,
 'currency_name': 'usd',
 'cik': '0000320193',
 'composite_figi': 'BBG000B9XRY4',
 'share_class_figi': 'BBG001S5N8V8',
 'market_cap': 2873542546320.0,
 'phone_number': '(408) 996-1010',
 'address': {'address1': 'ONE APPLE PARK WAY',
  'city': 'CUPERTINO',
  'state': 'CA',
  'postal_code': '95014'},
 'description': "Apple designs a wide variety of consumer electronic devices, including smartphones (iPhone), tablets (iPad), PCs (Mac), smartwatches (Apple Watch), AirPods, and TV boxes (Apple TV), among others. The iPhone makes up the majority of Apple's total revenue. In addition, Apple offers its customers a variety of services such as Apple Music, iCloud, Apple Care, Apple TV+, Apple Arcade, Apple Card, and Apple Pay, among others. Apple's products run internally developed software and semiconductors, and the firm is well known for its integration 

In [136]:
ipy.VBox(children=[
    ipy.Image(value=requests.get(data["branding"]["icon_url"]+f"?apiKey={key}").content, width=150),
    ipy.HBox(children=[
        ipy.HTML(value="<h1>{ticker}</h1>".format(ticker=data["ticker"])),
        ipy.HTML(value="<h2>{name}</h2>".format(name=data["name"])),
    ]),
    ipy.HTML(value="<p>{description}</p>".format(description=data["description"])),
    ipy.HBox(children=[
        ipy.FloatText(value=data["market_cap"], disabled=True),
    ]),
])